In [1]:
import os, sys
import feather
import re
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


# project_dir = os.path.dirname(os.path.dirname(os.path.abspath(os.path.curdir)))
project_dir = os.path.dirname(os.path.abspath(os.path.curdir))
new_path = os.path.join(project_dir, 'src')
sys.path.append(new_path)

import util as u
from model import pipeline as p

pd.options.display.max_columns = 100

/Users/kendra/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [10]:
import time

In [2]:
full_df = feather.read_dataframe('../data/processed/train_df.feather')

In [6]:
full_df.shape

(7449443, 27)

In [7]:
full_df['label'].value_counts(1)

benign      0.998858
phishing    0.001142
Name: label, dtype: float64

In [3]:
# take subset of data for initial model assessment
df = full_df.sample(frac=0.2).reset_index(drop=True)

In [9]:
df.shape

(1489889, 27)

In [10]:
# Make sure we've retained proportion of labels
df['label'].value_counts(1)

benign      0.998842
phishing    0.001158
Name: label, dtype: float64

In [4]:
del full_df

Convert int to float64

In [5]:
int_cols = df.select_dtypes(include='int').columns
int_cols = [col for col in int_cols if re.search('_ind', col) is None]

for col in int_cols:
    df[col] = df[col].astype(np.float64)

From EDA: Correlated features:
- length_url with length_path, url_X_cnt, and url_entropy
    - convert to fraction

In [6]:
cols_to_convert = ['length_path', 'length_domain', 'url_slash_cnt',
       'url_digit_cnt', 'url_special_char_cnt', 'url_reserved_char_cnt']

for col in cols_to_convert:
    new_col_name = col + '_frac_url_len'
    df[new_col_name] = df[col] / df['length_url'] 

In [12]:
# def extract_base_suffix(s):
#     if len(s) > 0:
#         return s.split('.')[0]
#     else:
#         return s

# df['base_suffix'] = df['suffix'].apply(extract_base_suffix)

### Split into X, y and a further train-test

In [7]:
target = 'label'
X = df.drop(columns=target)
y = df[target]

enc = LabelEncoder()
y_enc = enc.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.25)

# Preprocessing

In [8]:
all_cols = df.columns

target = 'label'

proc_dict = {
#     'base_suffix':[p.Consolidate(1), OneHotEncoder(handle_unknown='ignore')]
            }

num_cols = [col for col in all_cols if re.search('_cnt', col) is not None] + \
            ['length_url', 'hostname_entropy', 'url_entropy']

bool_cols = [col for col in all_cols if re.search('_ind', col) is not None]

pass_thru_cols = [col for col in all_cols if re.search('_frac_url_len', col) is not None]

for col in num_cols:
    proc_dict[col] = [StandardScaler()]

for col in bool_cols + pass_thru_cols:
    proc_dict[col] = [p.PassThrough()]

Do not include suffix, as benign only contains '.com', so this is a leaky variable of sorts

In [9]:
model_cols = ['subdomain_null_ind', 'subdomain_www_ind', 'length_url',
        'domain_dot_cnt', 'path_dot_cnt',
       'hostname_dash_cnt', 'url_hex_pattern_ind',
       'hostname_entropy', 'url_entropy', 'php_ind', 'abuse_ind', 'admin_ind',
       'verification_ind', 
              'length_path_frac_url_len',
       'length_domain_frac_url_len', 
       'url_slash_cnt_frac_url_len',
       'url_digit_cnt_frac_url_len', 'url_special_char_cnt_frac_url_len',
       'url_reserved_char_cnt_frac_url_len']

preproc_pipe = FeatureUnion(p.gen_pipeline(model_cols, proc_dict))

In [76]:
model = LogisticRegression(penalty='l2', random_state=19, solver='lbfgs')
lr_res = u.assess_model(preproc_pipe, model, X, y_enc, n=5)

/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data

/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by Stan

/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [77]:
lr_res

Precision-0                    0.998878
Recall-0 (Specificty)          0.999997
F1score-0                      0.999438
Precision-1                    0.912778
Recall-1 (Sensitivity)         0.018789
F1score-1                      0.036740
TN                        297636.400000
FN                           334.200000
FP                             0.800000
TP                             6.400000
AUC                            0.838422
Accuracy                       0.998876
dtype: float64

In [89]:
res_dict = {model: lr_res}

In [ ]:
# from importlib import reload

In [39]:
# reload(u)

<module 'util' from '/Users/kendra/Documents/data_science/Projects/phishing-urls/src/util.py'>

In [11]:
nb = GaussianNB()

nb_res = eval_model()

NameError: name 'nb' is not defined

In [53]:
nb_res

Precision-0                    0.999014
Recall-0 (Specificty)          0.996619
F1score-0                      0.997815
Precision-1                    0.026316
Recall-1 (Sensitivity)         0.085000
F1score-1                      0.040189
TN                        370815.000000
FN                           366.000000
FP                          1258.000000
TP                            34.000000
AUC                            0.852689
Accuracy                       0.995640
dtype: float64

In [45]:
model = rf

start_time = time.time()
rf_res = u.assess_model_no_cv(preproc_pipe, model, X_train, y_train, X_test, y_test)
duration = time.time() - start_time
print('Elapsed time: {} seconds'.format(duration))

TypeError: 'float' object is not callable

In [55]:
rf_res

Precision-0                    0.999162
Recall-0 (Specificty)          0.999954
F1score-0                      0.999558
Precision-1                    0.838095
Recall-1 (Sensitivity)         0.220000
F1score-1                      0.348515
TN                        372056.000000
FN                           312.000000
FP                            17.000000
TP                            88.000000
AUC                            0.871448
Accuracy                       0.999117
dtype: float64

## Fit more models

In [13]:
nb = GaussianNB()

lr_l2 = LogisticRegression(penalty='l2', random_state=19, solver='lbfgs')
# lr_l1 = LogisticRegression(penalty='l1', random_state=19, solver='saga') 
    # this didn't converge

# from sklearn.naive_bayes import MultinomialNB 
    # requires non-negative input
rf = RandomForestClassifier(n_estimators=100, max_depth=10)
# rf_ent = RandomForestClassifier(n_estimators=100, criterion='entropy')
# svm = SVC(class_weight='balanced') # add next time: , kernel='linear', gamma='scale') 
    # not efficient on large datasets

# knn = KNeighborsClassifier(n_neighbors=3)
    # this is O(n2)

In [14]:
res_dict = {}

In [15]:
# res_dict = {}
model_list = [lr_l2, nb, rf]
model_names = ['Log Reg', 'Naive Bayes', 'Random Forest']

model_dict = dict(zip(model_names, model_list))

for name, model in model_dict.items():
    print('Fitting {} ...'.format(name))
    start_time = time.time()
    res = u.assess_model_no_cv(preproc_pipe, model, X_train, y_train, X_test, y_test)
    duration = time.time() - start_time
    print('Elapsed time: {:.0f} seconds'.format(duration))
    res_dict[name] = res
    print('')

Fitting Log Reg ...


/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Elapsed time: 8 seconds

Fitting Naive Bayes ...
Elapsed time: 2 seconds

Fitting Random Forest ...
Elapsed time: 292 seconds



In [16]:
res_df = pd.DataFrame(res_dict)
res_df

,Log Reg,Naive Bayes,Random Forest
Precision-0,0.998859,0.998939,0.998878
Recall-0 (Specificty),1.000000,0.996968,0.999995
F1score-0,0.999429,0.997952,0.999436
Precision-1,1.000000,0.034247,0.888889
Recall-1 (Sensitivity),0.020737,0.092166,0.036866
F1score-1,0.040632,0.049938,0.070796
TN,372039.000000,370911.000000,372037.000000
FN,425.000000,394.000000,418.000000
FP,0.000000,1128.000000,2.000000
TP,9.000000,40.000000,16.000000


Random Forest performs the best (per F1-score)

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
RandomForestClassifier?

In [19]:
from sklearn.pipeline import make_pipeline

In [20]:
from sklearn.metrics import f1_score

In [27]:
from sklearn.metrics import make_scorer

In [23]:
pipe = make_pipeline(preproc_pipe, nb)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
f1_score(y_test, y_pred)

0.049937578027465665

In [ ]:
base_rf = RandomForestClassifier(n_estimators=100, max_depth=10)
pipe = make_pipeline(preproc_pipe, base_rf)
pipe.fit(X_train, y_train)
y_pred_test = pipe.predict(X_test)
y_pred_train = pipe.predict(X_train)

f1_train = 

In [28]:
f1_scorer = make_scorer(f1_score)

In [30]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='gini')
pipe = make_pipeline(preproc_pipe, rf)
pipe.fit(X_train, y_train)
y_pred_test = pipe.predict(X_test)
f1_score(y_test, y_pred_test)

0.0665188470066519

In [31]:
y_pred_train = pipe.predict(X_train)
f1_score(y_train, y_pred_train)

0.12700729927007298

In [32]:
import joblib
joblib.dump(pipe, '../models/rf_08_01.joblib')

['../models/rf_08_01.joblib']

In [35]:
pipe.steps

[('featureunion', FeatureUnion(n_jobs=None,
         transformer_list=[('subdomain_null_ind', Pipeline(memory=None,
       steps=[('select', FeatureSelector(feature_names=None)), ('subdomain_null_ind1', PassThrough())])), ('subdomain_www_ind', Pipeline(memory=None,
       steps=[('select', FeatureSelector(feature_names=None)), ('subdomain_www_ind1', PassT...t', FeatureSelector(feature_names=None)), ('url_reserved_char_cnt_frac_url_len1', PassThrough())]))],
         transformer_weights=None)),
 ('randomforestclassifier',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=10, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False))]

In [34]:
pipe.steps['randomforestclassifier']

TypeError: list indices must be integers or slices, not str

In [37]:
pipe.named_steps['randomforestclassifier']

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
pipe

In [38]:
params = {'randomforestclassifier__n_estimators':[50, 100, 200],
          'randomforestclassifier__max_depth':[5, 10, 30],
          'randomforestclassifier__criterion':['gini', 'entropy']}
rf = RandomForestClassifier(n_jobs=-1)
pipe = make_pipeline(preproc_pipe, rf)
clf = GridSearchCV(pipe, params, scoring=f1_scorer, cv=3, return_train_score=True)
clf.fit(X, y_enc)

/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: cannot label index with a null key

In [39]:
X_proc = preproc_pipe.fit_transform(X, y_enc)

In [ ]:
params = {'n_estimators':[50, 100, 200],
          'max_depth':[5, 10, 30],
          'criterion':['gini', 'entropy']}
rf = RandomForestClassifier(n_jobs=-1)
clf = GridSearchCV(rf, params, scoring=f1_scorer, cv=3, return_train_score=True)
clf.fit(X_proc, y_enc)

/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/kendra/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/kendra/anaconda3/lib/python3.6/site-packages/skle